In [131]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight



def augment_image(image):
    augmented_images = []
    augmented_images.append(cv2.flip(image, 1)) 
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols//2, rows//2), 15, 1)  
    augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))
    bright = np.clip(image * 1.2, 0, 1)
    augmented_images.append(bright)
    return augmented_images

def load_images_from_folder(folder, label, img_size=(64, 64), augment=False):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, img_size)
            img = img / 255.0
            images.append(img)
            labels.append(label)

            if augment:
                for aug_img in augment_image(img):
                    images.append(aug_img)
                    labels.append(label)

            print(f"✅ Loaded: {filename}, Label: {label}")
    return images, labels


first_images, first_labels = load_images_from_folder("./images/normal", 0, augment=True)
second_images, second_labels = load_images_from_folder("./images/wallpaper", 1, augment=True)

X = np.array(first_images + second_images )
y = np.array(first_labels + second_labels)


unique, counts = np.unique(y, return_counts=True)
print("Class Distribution:", dict(zip(unique, counts)))

X, y = shuffle(X, y, random_state=42)


✅ Loaded: Screenshot from 2024-07-22 10-08-09.png, Label: 0
✅ Loaded: shared image.jpeg, Label: 0
✅ Loaded: MicrosoftTeams-image.png, Label: 0
✅ Loaded: download.jpeg, Label: 0
✅ Loaded: wallpaper.png, Label: 1
Class Distribution: {np.int64(0): np.int64(16), np.int64(1): np.int64(4)}


In [132]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128, (3,3), activation='relu'), 
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(256, activation='relu'),  
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights_dict = dict(enumerate(class_weights))

model.fit(X, y, epochs=10, batch_size=32, validation_split=0.1, class_weight=class_weights_dict)
model.save("image_classifier.h5")


Epoch 1/10


/home/kunalbhande/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3889 - loss: 0.7107 - val_accuracy: 0.0000e+00 - val_loss: 1.4203
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.2222 - loss: 0.6347 - val_accuracy: 1.0000 - val_loss: 0.2948
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.7778 - loss: 0.5694 - val_accuracy: 1.0000 - val_loss: 0.4075
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 1.0000 - loss: 0.3963 - val_accuracy: 0.5000 - val_loss: 0.5944
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.8333 - loss: 0.3034 - val_accuracy: 0.5000 - val_loss: 0.4566
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 1.0000 - loss: 0.2180 - val_accuracy: 1.0000 - val_loss: 0.1402
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 1.0000 - loss: 0.1403 - val_accuracy: 1.0000 - val_loss: 0.0322
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 1.0000 - loss: 0.0930 - val_accuracy: 1.0000 - val_loss: 0.0188
Ep

In [133]:


model = load_model("image_classifier.h5")

def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return "Error: Unable to load image!"
    
    img = cv2.resize(img, (64, 64))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    

    prediction = model.predict(img)
    confidence = float(prediction[0][0])
    
    if confidence > 0.99 or confidence < 0.01:
        print("⚠️ WARNING: This image might be memorized!", prediction[0][0])
    else : print("⚠️ WARNING: This image might be not memorized!", prediction[0][0])
    
    return "wallpaper" if confidence > 0.5 else "normal"


print(predict_image("./images/wallpaper/wallpaper.png"))
print(predict_image("./images/normal/MicrosoftTeams-image.png"))
print(predict_image("../../../../Downloads/x.jpeg"))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
⚠️ WARNING: This image might be memorized! 0.99876684
wallpaper
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
⚠️ WARNING: This image might be memorized! 0.0002322834
normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
⚠️ WARNING: This image might be not memorized! 0.15453024
normal
